In [13]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from src.db.models import DbAuthority, DbWork, DbInstance, DbItem
from pyfuseki import FusekiUpdate, FusekiQuery
from pysolr import Solr
from src.schemas.settings import Settings

In [14]:
settings = Settings()

Clean DB

In [15]:
engine = create_engine(
     f"mariadb+mariadbconnector://{settings.db_user}:{settings.db_pass}@{settings.mariadb}:3306/bk")
session = scoped_session(
    sessionmaker(autocommit=False, autoflush=False, bind=engine))

In [16]:
session.query(DbAuthority).delete()
session.query(DbItem).delete()
session.query(DbInstance).delete()
session.query(DbWork).delete()
session.commit()

Clean Jena

In [17]:
fuseki = FusekiUpdate(settings.fuseki, 'bk') 

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""
response = fuseki.run_sparql(d)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [13]:
d = """CLEAR DEFAULT"""
response = fuseki.run_sparql(d)
response.convert()


{'statusCode': 200, 'message': 'Update succeeded'}

Clean Solr

In [18]:
solrAuthority = Solr(f'{settings.solr}/solr/authority/', timeout=10)
solrAuthority.delete(q="*:*",  commit=True)

solrCatalog = Solr(f'{settings.solr}/solr/catalog/', timeout=10)
solrCatalog.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">30</int>\n</lst>\n</response>\n'

In [10]:
results = solrCatalog.search(q="id:work#86", fl="*,[child]")
[doc] = results.docs
doc

{'id': 'work#86',
 'creationDate': ['2024-03-27T15:54:11Z'],
 'type': ['Work', 'Text'],
 'mainTitle': ['A Cidade Perdida dos Meninos-Peixes'],
 'language': ['Português'],
 'cdd': '546.5',
 'cutter': 'P567c',
 'isPartOf': ['Work'],
 '_version_': 1794968008062402560,
 'contribution': {'id': 'work#86/contribution/27',
  'agent': ['https://bibliokeia.com/authority/PersonalName/27'],
  'label': ['Pinto, Zemaria'],
  'role': ['http://id.loc.gov/vocabulary/relators/aut'],
  'roleLabel': 'Autor',
  '_version_': 1794968008062402560},
 'subject': {'id': 'work#86/subject/28',
  'type': ['Topic'],
  'uri': 'https://bibliokeia.com/authority/Topic/28',
  'label': ['Contos'],
  '_version_': 1794968008062402560}}

In [17]:
def is_dict(value):
    return type(value) == dict

In [24]:
doc_child = list(filter(is_dict, doc.values()))

In [28]:
ids = [doc['id'] for doc in doc_child]
ids.insert(0, 'work#86')
ids

['work#86', 'work#86/contribution/27', 'work#86/subject/28']

In [26]:
res = solrCatalog.delete(id=['work#86', 'work#86/contribution/27', 'work#86/subject/28'])
res

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">109</int>\n</lst>\n</response>\n'

In [5]:
w = DbWork(title="Titulo") 

In [7]:
w.id

In [8]:
session.add(w) 
session.commit()